In [2]:
import pandas as pd
import numpy as np

## Calculating Best Global Heroes

In [203]:
class best_Heroes:
    def __init__(self, data):
        self.data = data
        self.hero_stats = {}
        self._populate_hero_stats(mirrored = False)
        self.hero_categories = [
            {"name": "Ana", "category": ["Poke", "Brawl", "Dive"]},
            {"name": "Ashe", "category": ["Poke", "Brawl"]},
            {"name": "Bap", "category": ["Brawl", "Poke"]},
            {"name": "Bastion", "category": ["Poke", "Brawl"]},
            {"name": "Brig", "category": ["Dive"]},
            {"name": "Cass", "category": ["Brawl"]},
            {"name": "DVA", "category": ["Brawl", "Dive"]},
            {"name": "Doom", "category": ["Brawl", "Dive"]},
            {"name": "Echo", "category": ["Dive", "Brawl"]},
            {"name": "Genji", "category": ["Dive", "Brawl", "Poke"]},
            {"name": "Hanzo", "category": ["Poke"]},
            {"name": "Hazard", "category": ["Brawl"]},  
            {"name": "Illari", "category": ["Poke"]},  
            {"name": "Queen", "category": ["Brawl"]},
            {"name": "Junk", "category": ["Brawl"]},
            {"name": "Juno", "category": ["Dive", "Brawl"]},  
            {"name": "Kiri", "category": ["Dive", "Poke", "Brawl"]}, 
            {"name": "LW", "category": ["Dive"]},
            {"name": "Lucio", "category": ["Brawl", "Dive"]}, 
            {"name": "Mauga", "category": ["Brawl"]},
            {"name": "Mei", "category": ["Brawl", "Poke"]}, 
            {"name": "Mercy", "category": ["Poke"]},
            {"name": "Moira", "category": ["Dive", "Brawl"]},
            {"name": "Orisa", "category": ["Brawl", "Poke"]},
            {"name": "Pharah", "category": ["Poke"]},
            {"name": "Ram", "category": ["Brawl", "Poke"]},
            {"name": "Reaper", "category": ["Brawl", "Dive"]},
            {"name": "Rein", "category": ["Brawl"]},
            {"name": "Hog", "category": ["Poke"]},
            {"name": "Sigma", "category": ["Poke", "Brawl"]},
            {"name": "Soj", "category": ["Poke"]},  
            {"name": "Soldier", "category": ["Poke"]},
            {"name": "Sombra", "category": ["Dive"]},
            {"name": "Sym", "category": ["Brawl"]},
            {"name": "Torb", "category": ["Poke", "Brawl"]},  
            {"name": "Tracer", "category": ["Dive"]}, 
            {"name": "Venture", "category": ["Dive", "Brawl"]},  
            {"name": "Widow", "category": ["Poke"]},  
            {"name": "Winston", "category": ["Dive"]},
            {"name": "Ball", "category": ["Dive"]}, 
            {"name": "Zarya", "category": ["Brawl"]},  
            {"name": "Zen", "category": ["Poke"]}  
            ]
    def _populate_hero_stats(self, mirrored):
        # Iterate through the rows of the data to populate hero statistics
        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')


            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates
            
            for hero in all_heroes:
                if not mirrored and hero in allied_heroes and hero in enemy_heroes:
                    continue  

                if hero not in self.hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    self.hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                if hero in allied_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 1:
                        self.hero_stats[hero]['wins'] += 1
                if hero in enemy_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 0:
                        self.hero_stats[hero]['wins'] += 1
                
        # remove heroes with no matches
        self.hero_stats = {hero: stats for hero, stats in self.hero_stats.items() if stats['matches'] > 0}

    def get_bayesian_winrate(self, match_wins, hero_param=False, a=25, b=25):
        bayesian_winrates: dict[str, list[int]] = {}
        if hero_param:
            # Define a mapping for hero-specific (a, b) values
            hero_bayesian_params = {
            "Ana": (7, 4),  # Default Bayesian values for Ana
            "Ashe": (5, 6),
            "Bap": (0, 0),
            "Bastion": (0, 0),
            "Brig": (2, 1),
            "Cass": (0, 1),
            "DVA": (2, 1),
            "Doom": (0, 0),
            "Echo": (0, 0),
            "Genji": (2, 1),
            "Hanzo": (0, 0),
            "Hazard": (2, 1),
            "Illari": (0, 0),
            "Queen": (1, 0),
            "Junk": (0, 0),
            "Juno": (3, 4),
            "Kiri": (7, 2),
            "LW": (0, 0),
            "Lucio": (0, 0),
            "Mauga": (0, 2),
            "Mei": (4, 4),
            "Mercy": (0, 0),
            "Moira": (1, 0),
            "Orisa": (0, 0),
            "Pharah": (1, 0),
            "Ram": (2, 1),
            "Reaper": (1, 1),
            "Rein": (0, 0),
            "Hog": (0, 0),
            "Sigma": (0, 1),
            "Soj": (13, 8),
            "Soldier": (0, 0),
            "Sombra": (0, 0),
            "Sym": (0, 1),
            "Torb": (5, 3),
            "Tracer": (8, 12),
            "Venture": (0, 0),
            "Widow": (0, 0),
            "Winston": (10, 4),
            "Ball": (7, 6),
            "Zarya": (2, 2),
            "Zen": (0, 0)
        }
        for hero in match_wins:
            if hero_param:
                a, b = hero_bayesian_params[hero]
            matches = match_wins[hero]['matches']
            wins = match_wins[hero]['wins']
            bayesian_winrates[hero] = (wins + a) / (matches + a + b)  # Posterior mean of winrate
        
        return bayesian_winrates
    


    def normalise_winrates(self, winrates):
        """Normalises the winrates across roles."""
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        # Separate winrates by role
        for hero, winrate in winrates.items():
            role = self.hero_stats[hero]['Role']
            roles[role].append(winrate)
        
        normalised = {}
        
        # Normalise winrates within each role
        for role, winrate_list in roles.items():
            if not winrate_list:  # Skip empty roles
                continue
            winrate_array = np.array(winrate_list)
            min_winrate = np.min(winrate_array)
            max_winrate = np.max(winrate_array)
            
            for hero, winrate in winrates.items():
                if self.hero_stats[hero]['Role'] == role:
                    normalised[hero] = (winrate - min_winrate) / (max_winrate - min_winrate)
        
        # Remove heroes with roles that were empty
        empty_roles = [role for role, winrate_list in roles.items() if not winrate_list]
        for hero in list(normalised.keys()):
            if self.hero_stats[hero]['Role'] in empty_roles:
                del normalised[hero]
        
        return normalised

    def stat_visualiser(self, values, threshold = 0.9):
        # Separate heroes by role
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        for hero, value in values.items():
            role = self.hero_stats[hero]['Role']
            if value >= threshold:
                roles[role].append((hero, value))
        
        # Sort each role by value in descending order
        for role in roles:
            roles[role].sort(key=lambda x: x[1], reverse=True)
        
        # Print header
        print(f"{'Tank':<20} {'DPS':<20} {'Support':<20}")
        print("-" * 60)
        
        # Print heroes side by side
        for i in range(max(len(roles['Tank']), len(roles['Support']), len(roles['DPS']))):
            tank = f"{roles['Tank'][i][0]}: {roles['Tank'][i][1]:.4f}" if i < len(roles['Tank']) else ""
            dps = f"{roles['DPS'][i][0]}: {roles['DPS'][i][1]:.4f}" if i < len(roles['DPS']) else ""
            support = f"{roles['Support'][i][0]}: {roles['Support'][i][1]:.4f}" if i < len(roles['Support']) else ""            
            print(f"{tank:<20} {dps:<20} {support:<20}")


    def first_pick(self, teammates, curr_map, comp = "Poke,Brawl,Dive"):
        """ Calculate the total matches and wins of all heroes that were played on a given map or with specific teammates
        
        - Due to how Overwatch works, selecting your tank and both you supports at most (ignoring DPS) will provide best results.
        
        
        """

        comp_categories = set(comp.split(","))
        teammates = [hero.strip() for hero in teammates.split(',')] if isinstance(teammates, str) else teammates
        hero_stats: dict[str, list[int]] = {}

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',')
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')
            my_map = row['Map']

            # Check if any teammate is in allied heroes or if the map matches the current map
            self._synergy(teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories)


        for teammate in teammates:
            if teammate in hero_stats:
                del hero_stats[teammate]
        return hero_stats

    def _synergy(self, teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories):

        """ Synergy of my team with any hero pick by role. 

        - Currently uses the "OR" operation between map and teammates. Will need to be refined later
        
        
        
        
        
        """
        if (all(teammate in allied_heroes for teammate in teammates) or my_map == curr_map):
            both_conditions = (all(teammate in allied_heroes for teammate in teammates) or teammates == ['']) and my_map == curr_map

            for hero in allied_heroes:
                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue
                
                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                # Base increment
                hero_stats[hero]['matches'] += 1
                if result == 1:
                    hero_stats[hero]['wins'] += 1
                
                # Extra weighting when both conditions hold true
                if both_conditions:
                    hero_stats[hero]['matches'] += 1
                    if result == 1:
                        hero_stats[hero]['wins'] += 1


        if (all(enemy in enemy_heroes for enemy in teammates) or my_map == curr_map):
            both_conditions = (all(enemy in enemy_heroes for enemy in teammates) or teammates == ['']) and my_map == curr_map

            for hero in enemy_heroes:
                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue

                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                # Base increment
                hero_stats[hero]['matches'] += 1
                if result == 0:
                    hero_stats[hero]['wins'] += 1

                # Extra weighting when both conditions hold true
                if both_conditions:
                    hero_stats[hero]['matches'] += 1
                    if result == 0:
                        hero_stats[hero]['wins'] += 1


    def matchup_comparison(self, a_matchup, e_matchup):
        """Input a string of heroes for both a_matchup and e_matchup."""
        a_matchup = {hero.strip() for hero in a_matchup.split(",")}
        e_matchup = {hero.strip() for hero in e_matchup.split(",")}

        results = {
            'Placeholder': {'matches': 0, 'wins': 0, 'Role': 'Unknown'}
        }

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')

            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates

            if not((a_matchup.issubset(allied_heroes) and e_matchup.issubset(enemy_heroes)) or 
                   (a_matchup.issubset(enemy_heroes) and e_matchup.issubset(allied_heroes))):
                continue 

            results['Placeholder']['matches'] += 1   

            if (a_matchup.issubset(allied_heroes) and e_matchup.issubset(enemy_heroes) and result == 1) or \
               (a_matchup.issubset(enemy_heroes) and e_matchup.issubset(allied_heroes) and result == 0):
                results['Placeholder']['wins'] += 1

        return results


    def my_hero_performance(self):
        my_hero_perf: dict[str, list[int]] = {}
        for idx, row in self.data.iterrows():
            result = row['Result']
            my_hero = row['My_Hero']

            if my_hero in row['My_Tank'].split(','):
                my_role = 'Tank'
            elif my_hero in row['My_DPS'].split(','):
                my_role = 'DPS'
            elif my_hero in row['My_Sups'].split(','):
                my_role = 'Support'
            else:
                my_role = 'Unknown'

            if my_hero not in my_hero_perf:
                my_hero_perf[my_hero] = {'matches': 0, 'wins': 0, 'Role': my_role}

            my_hero_perf[my_hero]['matches'] += 1
            if result == 1:
                my_hero_perf[my_hero]['wins'] += 1

        return my_hero_perf
    

    def add_data(self, matches_1, matches_2):

        new_dataset = {}

        for hero_1 in matches_1:
            if hero_1 in matches_2:
                new_dataset[hero_1] = {'matches': matches_1[hero_1]['matches'] + matches_2[hero_1]['matches'], 
                                        'wins': matches_1[hero_1]['wins'] + matches_2[hero_1]['wins'], 'Role':matches_1[hero_1]['Role']}
        return new_dataset
            
            

    def matchup_hero_prob_winrate(self, a_tank, a_DPS, a_supports, e_tank, e_DPS, e_supports):
        """Calculate the win probability based on individual hero matchups."""
        hero_winrate_prob = {}

        for idx, row in self.data.iterrows():
            result = row['Result']
            team1_tank, team1_dps, team1_sups = row['My_Tank'], row['My_DPS'], row['My_Sups']
            team2_tank, team2_dps, team2_sups = row['E_Tank'], row['E_DPS'], row['E_Sups']
            team1_dps_list = team1_dps.split(', ') if team1_dps else []
            team1_sups_list = team1_sups.split(', ') if team1_sups else []
            team2_dps_list = team2_dps.split(', ') if team2_dps else []
            team2_sups_list = team2_sups.split(', ') if team2_sups else []

            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')
            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates

            # Check for specific hero matchups
            tank_match = (a_tank and e_tank and ((team1_tank == a_tank and team2_tank == e_tank) or (team2_tank == a_tank and team1_tank == e_tank)))
            dps_match = (a_DPS and e_DPS and ((set(team1_dps_list) == set(a_DPS.split(', ')) and set(team2_dps_list) == set(e_DPS.split(', '))) or (set(team2_dps_list) == set(a_DPS.split(', ')) and set(team1_dps_list) == set(e_DPS.split(', ')))))
            supp_match = (a_supports and e_supports and ((set(team1_sups_list) == set(a_supports.split(', ')) and set(team2_sups_list) == set(e_supports.split(', '))) or (set(team2_sups_list) == set(a_supports.split(', ')) and set(team1_sups_list) == set(e_supports.split(', ')))))
            if tank_match or dps_match or supp_match:
                for hero in all_heroes:
                    # Initialize hero data if not already done
                    if hero not in hero_winrate_prob:
                        if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                            role = 'Tank'
                        elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                            role = 'DPS'
                        elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                            role = 'Support'
                        else:
                            role = 'Unknown'
                        hero_winrate_prob[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                    # Only count hero stats if it's from the relevant team
                    if hero in allied_heroes:
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 1:  # win for the allied team
                            hero_winrate_prob[hero]['wins'] += 1
                    elif hero in enemy_heroes:
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 0:  # win for the enemy team
                            hero_winrate_prob[hero]['wins'] += 1

        return hero_winrate_prob





df = pd.read_csv('stats.csv')
df = df[df['Closeness'] == 0]  # Keep only rows where closeness is 0        
heroes = best_Heroes(df)
hero_prob = heroes.matchup_hero_prob_winrate("", "", "Brig,Kiri", "", "", "Brig,Juno")
print(hero_prob)
bayesian_prob = heroes.get_bayesian_winrate(hero_prob)
normalised_prob = heroes.normalise_winrates(bayesian_prob)
heroes.stat_visualiser(normalised_prob, 0)



# first_pick = heroes.first_pick("Orisa", "")
# bayesian_winrates = heroes.get_bayesian_winrate(first_pick, True)
# normalised = heroes.normalise_winrates(bayesian_winrates)
# heroes.stat_visualiser(normalised, 0)


# bayesian_standard = heroes.get_bayesian_winrate(heroes.hero_stats, True)
# normalised_standard = heroes.normalise_winrates(bayesian_standard)
# heroes.stat_visualiser(normalised_standard, 0)





{'DVA': {'matches': 1, 'wins': 0, 'Role': 'Tank'}, 'Tracer': {'matches': 1, 'wins': 1, 'Role': 'DPS'}, 'Sombra': {'matches': 1, 'wins': 0, 'Role': 'DPS'}, 'Juno': {'matches': 1, 'wins': 0, 'Role': 'Support'}, 'Kiri': {'matches': 1, 'wins': 1, 'Role': 'Support'}, 'Reaper': {'matches': 1, 'wins': 0, 'Role': 'DPS'}, 'Ball': {'matches': 1, 'wins': 1, 'Role': 'Tank'}, 'Brig': {'matches': 1, 'wins': 1, 'Role': 'Support'}, 'Genji': {'matches': 1, 'wins': 1, 'Role': 'DPS'}}
Tank                 DPS                  Support             
------------------------------------------------------------
Ball: 1.0000         Tracer: 1.0000       Kiri: 1.0000        
DVA: 0.0000          Genji: 1.0000        Brig: 1.0000        
                     Sombra: 0.0000       Juno: 0.0000        
                     Reaper: 0.0000                           
